In [1]:
import pandas as pd

In [2]:
# Load the datasets
indian_food_df = pd.read_csv("indian_food.csv")
good_ingredients_mood_df = pd.read_csv("ingredients_mood-good.csv")
bad_ingredients_mood_df = pd.read_csv("ingredients_mood-bad.csv")

In [25]:
# Example to filter based on mood to get relevant ingredients
def get_dishes_based_on_mood(mood):
    # Make empty dataframes for positive and negative dishes
    positive_dishes = pd.DataFrame(columns=indian_food_df.columns)
    negative_dishes = pd.DataFrame(columns=indian_food_df.columns)

    # Get all ingredients that have a non NaN value in the respective mood column
    positive_ingredients = good_ingredients_mood_df[good_ingredients_mood_df[mood].notna()]
    negative_ingredients = bad_ingredients_mood_df[bad_ingredients_mood_df[mood].notna()]

    # Get dishes to recommend from the positive_ingredients df based on the mood
    for index, row in indian_food_df.iterrows():
        for ingredient in positive_ingredients["ingredients"]:
            if ingredient.lower() in row["ingredients"].lower():
                positive_dishes = pd.concat([positive_dishes, indian_food_df[indian_food_df["name"] == row["name"]]])
        for ingredient in negative_ingredients["ingredients"]:
            if ingredient.lower() in row["ingredients"].lower():
                negative_dishes = pd.concat([positive_dishes, indian_food_df[indian_food_df["name"] == row["name"]]])
    
    # Remove duplicate rows in both dataframes
    positive_dishes = positive_dishes.drop_duplicates()
    negative_dishes = negative_dishes.drop_duplicates()

    return positive_dishes, negative_dishes


In [26]:
get_dishes_based_on_mood("Disgust")[0]

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
11,Lassi,"Yogurt, milk, nuts, sugar",vegetarian,5,5,sweet,dessert,Punjab,North
38,Bandar laddu,"Besan, jaggery, cardamom powder, ghee, cashews...",vegetarian,5,35,sweet,dessert,Andhra Pradesh,South
64,Maach Jhol,"Fish, potol, tomato, chillies, ginger, garlic",non vegetarian,10,40,spicy,main course,Assam,North East
...,...,...,...,...,...,...,...,...,...
239,Koldil Chicken,"Banana flower, chicken, green chili, mustard o...",non vegetarian,-1,-1,spicy,main course,Assam,North East
242,Masor Koni,"Fish roe, pumpkin flowers, mustard oil, turmer...",non vegetarian,-1,-1,spicy,main course,Assam,North East
243,Mishti Chholar Dal,"Chana dal, fresh coconut, ginger, cinnamon, ra...",vegetarian,10,30,sweet,main course,West Bengal,East
247,Prawn malai curry,"Coconut milk, prawns, garlic, turmeric, sugar",non vegetarian,15,50,spicy,main course,West Bengal,East


In [5]:
def recommend_dishes(mood, allergies, is_vegan, like_spicy):
    # Get relevant ingredients for the mood
    positive_dishes, negative_dishes = get_dishes_based_on_mood(mood)
    positive_ingredients = good_ingredients_mood_df[good_ingredients_mood_df[mood].notna()]
    negative_ingredients = bad_ingredients_mood_df[bad_ingredients_mood_df[mood].notna()]

    # Exclude based on user restrictions
    if is_vegan:
        positive_dishes = positive_dishes[positive_dishes["diet"] == "vegetarian"]
    for allergen in allergies:
        positive_dishes = positive_dishes[~positive_dishes['ingredients'].str.contains(allergen, case=False)]
    if not like_spicy:
        positive_dishes = positive_dishes[~positive_dishes['flavor_profile'].str.contains("spicy", case=False)]

    final_recom = positive_dishes.sample(n=3)  # For simplicity, randomly pick three satisfying all conditions

    # Check if final_recom is empty
    if final_recom.empty:
        print("Sorry, I couldn't find any dishes satisfying your preferences and restrictions.")
    else:
        print(f"Based on your mood and preferences, I recommend trying the following dishes: ")
        for index, row in final_recom.iterrows():
            print(f"{row['name']} from the {row['state']} state of the {row['region']} region, which is a {row['course']} dish.")
            print(f"{row['name']} is a {row['flavor_profile']} dish and is {row['diet']}.")
            print(f"{row['name']} contains the ingredients {row['ingredients']}.")
            for ingredient in row['ingredients'].split(","):
                print(positive_ingredients.loc[positive_ingredients['ingredients'].lower() == ingredient.lower()]["Y Reason"])


In [7]:
indian_food_df.head()

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


In [8]:
good_ingredients_mood_df.head()

,ingredients,Sad,Anger,Disgust,Fear,Joy,Neutral,Y Reason,Source,translation
0,elachi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cardamom
1,vegetable oil,y,NaN,NaN,NaN,NaN,NaN,Vegetable oil contains vitamin E which has bee...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,NaN
2,ghee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,jaggery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
4,Rice flour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
